# Overview of [smolagents](https://huggingface.co/docs/smolagents)

In [1]:
import os

from smolagents import LiteLLMModel, ToolCallingAgent

from travel_agent.qdrant import client as qdrant_client
from travel_agent.retrieval.embedding.embedding_generation import MODELS_PROMPTS
from travel_agent.retrieval.smolagents.prompts import RUS_PROMPT_TEMPLATES
from travel_agent.retrieval.smolagents.tool import TravelReviewQueryTool

if not os.getenv("QDRANT_URL"):
    print("Set QDRANT_URL and QDRANT_API_KEY env")
    exit(1)

print(f"Available embed models:\n- {'\n- '.join(MODELS_PROMPTS.keys())}")

2025-04-20 13:02:20.612 | DEBUG    | travel_agent.qdrant.client:create_client:12 - Connecting to Qdrant at https://21a1c8c4-0d63-4176-9930-ee7848dafd2f.eu-central-1-0.aws.cloud.qdrant.io
2025-04-20 13:02:21.014 | SUCCESS  | travel_agent.qdrant.client:create_client:22 - connected: title='qdrant - vector search engine' version='1.13.6' commit='4db98ecdbf66bedbce268ec26865ea446d0bba7b'


Available embed models:
- cointegrated/rubert-tiny2
- DeepPavlov/rubert-base-cased-sentence
- ai-forever/sbert_large_nlu_ru
- ai-forever/sbert_large_mt_nlu_ru
- sentence-transformers/distiluse-base-multilingual-cased-v1
- sentence-transformers/distiluse-base-multilingual-cased-v2
- sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
- sentence-transformers/paraphrase-multilingual-mpnet-base-v2
- intfloat/multilingual-e5-large
- intfloat/multilingual-e5-base
- intfloat/multilingual-e5-small
- ai-forever/ru-en-RoSBERTa
- sergeyzh/BERTA


In [2]:
# Replace with your favourite local model
MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0"
model_id = f"ollama_chat/{MODEL_NAME}"


model = LiteLLMModel(
    model_id=model_id,
    api_base=os.getenv("OLLAMA_URL", "http://127.0.0.1:11434"),
    num_ctx=8192,  # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
)

test_response = model([{"role": "user", "content": [{"text": "Алло, это Кизару у телефона?", "type": "text"}]}]).content
print(test_response)
assert test_response

Привет! Да, я здесь. - Это Кизару. Как я могу помочь вам сегодня?


In [3]:
embed_model_name = "intfloat/multilingual-e5-base"
collection_name = "moskva_intfloat_multilingual_e5_base"


review_search_tool = TravelReviewQueryTool(embed_model_name, qdrant_client, collection_name)


agent = ToolCallingAgent(tools=[review_search_tool], model=model, max_steps=3, verbosity_level=2)

2025-04-20 13:02:27.964 | INFO     | travel_agent.retrieval.smolagents.tool:__init__:70 - Using device: mps


In [4]:
print(agent.system_prompt)

You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer"

In [5]:
# Sanity check
agent.run("Что ты думаешь о римской империи?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Что ты думаешь о римской империи?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
  "name": "final_answer",                                                                                          
  "arguments": {                                                                                                   
    "answer": "Римская империя была одним из величайших государств в истории человечества, известным своими        
значительными достижениями в области права, архитектуры, инженерии и культуры. Она оставила после себя наследие в  
виде правовых систем, мостов, дорог и искусства, которое до сих пор оказывает влияние на современный мир."         
  }                                                                                                                
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Римская империя была одним из величайших государств в  │
│ истории человечества, известным своими значительными достижениями в области права, архитектуры, инженерии и     │
│ культуры. Она оставила после себя наследие в виде правовых систем, мостов, дорог и искусства, которое до сих    │
│ пор оказывает влияние на современный мир.'}                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Римская империя была одним из величайших государств в истории человечества, известным своими 
значительными достижениями в области права, архитектуры, инженерии и культуры. Она оставила после себя наследие в 
виде правовых систем, мостов, дорог и искусства, которое до сих пор оказывает влияние на современный мир.

[Step 1: Duration 14.08 seconds| Input tokens: 1,431 | Output tokens: 99]

'Римская империя была одним из величайших государств в истории человечества, известным своими значительными достижениями в области права, архитектуры, инженерии и культуры. Она оставила после себя наследие в виде правовых систем, мостов, дорог и искусства, которое до сих пор оказывает влияние на современный мир.'

In [4]:
resp = agent.run("Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?                                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
  "name": "travel_review_query",                                                                                   
  "arguments": {                                                                                                   
    "query": "посоветуй хороший японский ресторан в Москве",                                                       
    "min_rating": 4                                                                                                
  }                                                                                                                
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'посоветуй хороший японский ресторан в Москве',   │
│ 'min_rating': 4}                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-04-20 13:02:38.674 | DEBUG    | travel_agent.retrieval.smolagents.tool:forward:107 - filters: [FieldCondition(key='rating', match=None, range=Range(lt=None, gt=None, gte=4.0, lte=None), geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)]


...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{ "name": "final_answer", "arguments": {"answer": "На основе отзывов рекомендую посетить следующие места:\n\n-     
**Изюми** (Мясницкая улица, 38с1) - отличный выбор для спокойной встречи с друзьями и разнообразного меню. Особенно
стоит отметить аутентичную кухню японского шефа.\n- **Tsunami Sushi & Cocktails** (Благовещенский переулок, 1А) -  
подойдет тем, кто ценит атмосферу ночного Токио и разнообразие вкусов в роллах. Отличный выбор для вечернего       
посещения.\n\nОба ресторана заслуживают высоких оценок и рекомендаций по качеству еды."}}                          

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'На основе отзывов рекомендую посетить следующие        │
│ места:\n\n- **Изюми** (Мясницкая улица, 38с1) - отличный выбор для спокойной встречи с друзьями и               │
│ разнообразного меню. Особенно стоит отметить аутентичную кухню японского шефа.\n- **Tsunami Sushi & Cocktails** │
│ (Благовещенский переулок, 1А) - подойдет тем, кто ценит атмосферу ночного Токио и разнообразие вкусов в роллах. │
│ Отличный выбор для вечернего посещения.\n\nОба ресторана заслуживают высоких оценок и рекомендаций по качеству  │
│ еды.'}                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: На основе отзывов рекомендую посетить следующие места:

- **Изюми** (Мясницкая улица, 38с1) - отличный выбор для спокойной встречи с друзьями и разнообразного меню. 
Особенно стоит отметить аутентичную кухню японского шефа.
- **Tsunami Sushi & Cocktails** (Благовещенский переулок, 1А) - подойдет тем, кто ценит атмосферу ночного Токио и 
разнообразие вкусов в роллах. Отличный выбор для вечернего посещения.

Оба ресторана заслуживают высоких оценок и рекомендаций по качеству еды.

[Step 2: Duration 28.73 seconds| Input tokens: 4,603 | Output tokens: 228]

In [7]:
print(resp)

Основываясь на полученных отзывах, рекомендую посетить ресторан 'Izumi' по адресу Мясницкая улица, д.38с1 в Москве. Отзывы отмечают разнообразие блюд, качественное обслуживание и приятную атмосферу. Также стоит обратить внимание на 'Tsunami Sushi & Cocktails', расположенный по адресу Благовещенский переулок, д.1А, особенно если вы цените не только еду, но и современную атмосферу.


In [8]:
agent.memory.steps

[TaskStep(task='Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?', task_images=None),
 ActionStep(model_input_messages=[{'role': <MessageRole.SYSTEM: 'system'>, 'content': [{'type': 'text', 'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\

In [5]:
# translated prompts

agent = ToolCallingAgent(
    tools=[review_search_tool],
    model=model,
    prompt_templates=RUS_PROMPT_TEMPLATES,
    max_steps=3,
    verbosity_level=2,  # DEBUG
)

print(agent.system_prompt)

Вы — эксперт-ассистент, который может решать любые задачи с использованием вызовов инструментов. Вам будет дана задача, которую вы должны решить наилучшим образом.
Для этого вам предоставлен доступ к некоторым инструментам.
Вызов инструмента, который вы пишете, является действием: после выполнения инструмента вы получите результат вызова инструмента как "наблюдение".
Эта последовательность Действие/Наблюдение может повторяться N раз, вам следует выполнять несколько шагов, когда это необходимо.
Вы можете использовать результат предыдущего действия как ввод для следующего действия.
Наблюдение всегда будет строкой: она может представлять файл, например "image_1.jpg".
Затем вы можете использовать его как ввод для следующего действия. Вы можете сделать это, например, следующим образом:

Наблюдение: "image_1.jpg"

Действие:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}
Чтобы предоставить окончательный ответ на задачу, используйте блок действия с "name": "final_an

In [6]:
resp = agent.run("Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Я бы хотел посетить японский ресторан. Можешь ли посоветовать хороший в Москве?                                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
 "name": "travel_review_query",                                                                                    
 "arguments": {                                                                                                    
   "query": "посоветуй хороший японский ресторан в Москве",                                                        
   "min_rating": 4                                                                                                 
 }                                                                                                                 
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'посоветуй хороший японский ресторан в Москве',   │
│ 'min_rating': 4}                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-04-20 13:03:30.822 | DEBUG    | travel_agent.retrieval.smolagents.tool:forward:107 - filters: [FieldCondition(key='rating', match=None, range=Range(lt=None, gt=None, gte=4.0, lte=None), geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)]


...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{ "name": "final_answer", "arguments": {"answer": "Из двух предложенных вариантов на основе отзывов можно          
рекомендовать:\n\n1️⃣ **Izumi** (ул. Мясницкая, 38с1) — отмечен за отличное разнообразие блюд и адекватную цену,     
приятный интерьер и удобные функции вызова официанта.\n- Рейтинг: 🌟🌟🌟🌟🌟\n\n2️⃣ **Tsunami Sushi & Cocktails**    
(Благовещенский переулок, 1А) — выделяется неоновой атмосферой Токио и отличными коктейлями, рекомендован          
посетителями как идеальное место для ужина.\n- Рейтинг: 🌟🌟🌟🌟🌟\n\nОба места получили высокие оценки и подойдут 
для приятного вечера с друзьями."} }                                                                               

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Из двух предложенных вариантов на основе отзывов можно │
│ рекомендовать:\n\n1️⃣ **Izumi** (ул. Мясницкая, 38с1) — отмечен за отличное разнообразие блюд и адекватную цену,  │
│ приятный интерьер и удобные функции вызова официанта.\n- Рейтинг: 🌟🌟🌟🌟🌟\n\n2️⃣ **Tsunami Sushi & Cocktails** │
│ (Благовещенский переулок, 1А) — выделяется неоновой атмосферой Токио и отличными коктейлями, рекомендован       │
│ посетителями как идеальное место для ужина.\n- Рейтинг: 🌟🌟🌟🌟🌟\n\nОба места получили высокие оценки и       │
│ подойдут для приятного вечера с друзьями.'}                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Из двух предложенных вариантов на основе отзывов можно рекомендовать:

1️⃣ **Izumi** (ул. Мясницкая, 38с1) — отмечен за отличное разнообразие блюд и адекватную цену, приятный интерьер и 
удобные функции вызова официанта.
- Рейтинг: 🌟🌟🌟🌟🌟

2️⃣ **Tsunami Sushi & Cocktails** (Благовещенский переулок, 1А) — выделяется неоновой атмосферой Токио и отличными 
коктейлями, рекомендован посетителями как идеальное место для ужина.
- Рейтинг: 🌟🌟🌟🌟🌟

Оба места получили высокие оценки и подойдут для приятного вечера с друзьями.

[Step 2: Duration 34.93 seconds| Input tokens: 4,804 | Output tokens: 290]

In [10]:
resp = agent.run("Подскажи мне ателье по ременту обуви?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Подскажи мне ателье по ременту обуви?                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q4_0 ───────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{"name": "travel_review_query", "arguments": {"query": "ателье по ремонту обуви", "rubrics": null, "min_rating":   
4}}                                                                                                                

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'travel_review_query' with arguments: {'query': 'ателье по ремонту обуви', 'rubrics': None,       │
│ 'min_rating': 4}                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-04-20 13:05:42.351 | DEBUG    | travel_agent.retrieval.smolagents.tool:forward:107 - filters: [FieldCondition(key='rating', match=None, range=Range(lt=None, gt=None, gte=4.0, lte=None), geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)]


...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
{                                                                                                                  
"name": "final_answer",                                                                                            
"arguments": {                                                                                                     
    "answer": "Наиболее рекомендованные места по отзывам с рейтингом 5.0 и специализацией на ремонте обуви:\n\n1.  
Люкс мастер - Москва, Кожевническая улица, 10, стр. 1\n   Рейтинг: 5.0\n   Отзыв: «Пользуюсь услугами ремонта обуви
и ателье уже почти год. Всем довольна...»\n\n2. Дом Быта - Москва, Зарайская улица, 53\n   Рейтинг: 5.0\n   Отзыв: 
«Отличное Ателье! Пожалуй, самое лучшее соотношение качества и цены в нашем районе...»\n\n3. Косино Парк (Дом быта)
- Москва, Святоозёрская улица, 1А\n   Рейтинг: 5.0\n   Отзыв: «Ремонт обуви делают очень быстро и качественно.     
Мастер раньше работал на Рязанском проспекте...»\n\nВыбирайте любое из этих ателье в зависимости от вашего района  
проживания."                                                                                                       
}                                                                                                                  
}                                                                                                                  

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Наиболее рекомендованные места по отзывам с рейтингом  │
│ 5.0 и специализацией на ремонте обуви:\n\n1. Люкс мастер - Москва, Кожевническая улица, 10, стр. 1\n   Рейтинг: │
│ 5.0\n   Отзыв: «Пользуюсь услугами ремонта обуви и ателье уже почти год. Всем довольна...»\n\n2. Дом Быта -     │
│ Москва, Зарайская улица, 53\n   Рейтинг: 5.0\n   Отзыв: «Отличное Ателье! Пожалуй, самое лучшее соотношение     │
│ качества и цены в нашем районе...»\n\n3. Косино Парк (Дом быта) - Москва, Святоозёрская улица, 1А\n   Рейтинг:  │
│ 5.0\n   Отзыв: «Ремонт обуви делают очень быстро и качественно. Мастер раньше работал на Рязанском              │
│ проспекте...»\n\nВыбирайте любое из этих ателье в зависимости от вашего района проживания.'}                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Наиболее рекомендованные места по отзывам с рейтингом 5.0 и специализацией на ремонте обуви:

1. Люкс мастер - Москва, Кожевническая улица, 10, стр. 1
   Рейтинг: 5.0
   Отзыв: «Пользуюсь услугами ремонта обуви и ателье уже почти год. Всем довольна...»

2. Дом Быта - Москва, Зарайская улица, 53
   Рейтинг: 5.0
   Отзыв: «Отличное Ателье! Пожалуй, самое лучшее соотношение качества и цены в нашем районе...»

3. Косино Парк (Дом быта) - Москва, Святоозёрская улица, 1А
   Рейтинг: 5.0
   Отзыв: «Ремонт обуви делают очень быстро и качественно. Мастер раньше работал на Рязанском проспекте...»

Выбирайте любое из этих ателье в зависимости от вашего района проживания.

[Step 2: Duration 33.42 seconds| Input tokens: 4,221 | Output tokens: 317]